# **Diabetic Patient Readmission: Modeling**

This dataset was analyzed by numerous Virginia Commonwealth University faculty in a recent research article which is accompanied by feature descriptions. These can be found at https://www.hindawi.com/journals/bmri/2014/781670/tab1/.

In [19]:
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV

from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.feature_selection import chi2
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

from timeit import default_timer as timer

%matplotlib inline

np.random.seed(42)

In [2]:
df1 = pd.read_csv("clean_data2.csv")
df1.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,Pediatrics-Endocrinology,...,No,No,No,No,No,No,No,No,No,Other
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,missing,...,No,Up,No,No,No,No,No,Ch,Yes,Other
2,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,missing,...,No,Up,No,No,No,No,No,Ch,Yes,Other
3,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,missing,...,No,Steady,No,No,No,No,No,Ch,Yes,Other
4,35754,82637451,Caucasian,Male,[50-60),2,1,2,3,missing,...,No,Steady,No,No,No,No,No,No,Yes,Other


# Pre-processing:

In [3]:
df1 = df1.drop(columns=['encounter_id','patient_nbr'])    #irrelevant for modeling

In [4]:
X = df1.drop(columns=['readmitted'])
y = df1[['readmitted']].values.ravel()
X.shape, y.shape

((64781, 42), (64781,))

In [5]:
X = pd.get_dummies(X, drop_first=True)
X.head()

,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_diagnoses,...,insulin_Steady,insulin_Up,glyburide-metformin_No,glyburide-metformin_Steady,glyburide-metformin_Up,glipizide-metformin_Steady,metformin-rosiglitazone_Steady,metformin-pioglitazone_Steady,change_No,diabetesMed_Yes
0,6,25,1,1,41,0,1,0,0,1,...,0,0,1,0,0,0,0,0,1,0
1,1,1,7,3,59,0,18,0,0,9,...,0,1,1,0,0,0,0,0,0,1
2,1,1,7,2,44,1,16,0,0,7,...,0,1,1,0,0,0,0,0,0,1
3,1,1,7,1,51,0,8,0,0,5,...,1,0,1,0,0,0,0,0,0,1
4,2,1,2,3,31,6,16,0,0,9,...,1,0,1,0,0,0,0,0,1,1


In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

4 seconds elapsed.


# Train-test split:

In [7]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.25, random_state=42)

In [8]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(48585, 2298) (16196, 2298) (48585,) (16196,)


# Model Tuning:

**K-Nearest Neighbors:**

In [9]:
start = timer()
knn = KNeighborsClassifier()
param_dist = {'n_neighbors': np.arange(1,20),
              'leaf_size': np.arange(1,30),
              'p': [1,2]}
knn_cv = RandomizedSearchCV(knn, param_dist, cv=5, verbose=1, n_jobs=-1)
knn_cv = knn_cv.fit(X_train, y_train)
end = timer()
print(f'{round(end - start)} seconds elapsed.')

Fitting 5 folds for each of 10 candidates, totalling 50 fits
6667 seconds elapsed.


In [10]:
print("Best Score:" + str(knn_cv.best_score_))
print("Best Parameters: " + str(knn_cv.best_params_))

Best Score:0.9158999691262736
Best Parameters: {'p': 2, 'n_neighbors': 18, 'leaf_size': 23}


In [23]:
start = timer()
best_knn = KNeighborsClassifier(n_neighbors=18, leaf_size=23, p=2, n_jobs=-1)
best_knn = best_knn.fit(X_train, y_train)

print('train:', best_knn.score(X_train, y_train), 'test:', best_knn.score(X_test, y_test))
end = timer()
print(f'{round(end-start)} seconds elapsed.')

train: 0.9159617165791911 test: 0.9152259817238825
1503 seconds elapsed.


In [24]:
y_pred = best_knn.predict(X_test)
print(classification_report(y_test, y_pred, labels=best_knn.classes_))

              precision    recall  f1-score   support

         <30       0.50      0.00      0.00      1373
       Other       0.92      1.00      0.96     14823

    accuracy                           0.92     16196
   macro avg       0.71      0.50      0.48     16196
weighted avg       0.88      0.92      0.88     16196



In [25]:
auc_score = roc_auc_score(y_test, best_knn.predict_proba(X_test)[:,1])
print('AUC:', auc_score)

AUC: 0.5532298112139366


**Random Forest:**

In [11]:
start = timer()
max_depth = np.arange(10, 100, 10)
max_depth = np.append(max_depth, None)
rfclf = RandomForestClassifier()
param_dist = {'max_depth': max_depth,
              'max_features':['auto','sqrt'],
              'min_samples_leaf': [1, 2, 4],
              'min_samples_split': [2, 5, 10],
              'n_estimators': np.arange(200, 2000, 200)}
rfclf_cv = RandomizedSearchCV(rfclf, param_dist, cv=5, verbose=10, n_jobs=-1)
rfclf_cv = rfclf_cv.fit(X_train, y_train)
end = timer()
print(f'{round(end - start)} seconds elapsed.')

Fitting 5 folds for each of 10 candidates, totalling 50 fits
8996 seconds elapsed.


In [12]:
print("Best Score:" + str(rfclf_cv.best_score_))
print("Best Parameters: " + str(rfclf_cv.best_params_))

Best Score:0.9159822990634969
Best Parameters: {'n_estimators': 400, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 30}


In [20]:
start = timer()
best_rf = RandomForestClassifier(n_estimators=400, min_samples_split=10, min_samples_leaf=1,
                                  max_features='auto', max_depth=30, n_jobs=-1)
best_rf = best_rf.fit(X_train, y_train)

print('train:', best_rf.score(X_train, y_train), 'test:', best_rf.score(X_test, y_test))
end = timer()
print(f'{round(end-start)} seconds elapsed.')

train: 0.9159822990634969 test: 0.9152259817238825
163 seconds elapsed.


In [21]:
y_pred = best_rf.predict(X_test)
print(classification_report(y_test, y_pred, labels=best_rf.classes_))

              precision    recall  f1-score   support

         <30       0.00      0.00      0.00      1373
       Other       0.92      1.00      0.96     14823

    accuracy                           0.92     16196
   macro avg       0.46      0.50      0.48     16196
weighted avg       0.84      0.92      0.87     16196



In [22]:
auc_score = roc_auc_score(y_test, best_rf.predict_proba(X_test)[:,1])
print('AUC:', auc_score)

AUC: 0.6281197027571618


**XGBoost:**

In [13]:
start = timer()
xgbclf = xgb.XGBClassifier(scale_pos_weight=1)
param_dist = {'learning_rate': np.arange(0.0, 1.0, 0.1),
              'n_estimators': np.arange(100, 1000, 100),
              'max_depth': np.arange(3, 13, 2),
              'min_child_weight': np.arange(1, 10, 1),
              'gamma': np.arange(0.0, 1.0, 0.1),
              'objective': ['binary:logistic', 'reg:linear']}
xgbclf_cv = RandomizedSearchCV(xgbclf, param_dist, cv=5, verbose=1, n_jobs=-1)
xgbclf_cv = xgbclf_cv.fit(X_train, y_train)
end = timer()
print(f'{round(end - start)} seconds elapsed.')

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[08:49:18] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
26345 seconds elapsed.


In [14]:
print("Best Score:" + str(xgbclf_cv.best_score_))
print("Best Parameters: " + str(xgbclf_cv.best_params_))

Best Score:0.9155912318616857
Best Parameters: {'objective': 'reg:linear', 'n_estimators': 600, 'min_child_weight': 4, 'max_depth': 9, 'learning_rate': 0.2, 'gamma': 0.7000000000000001}


In [26]:
start = timer()
best_xgb = xgb.XGBClassifier(objective='reg:linear', n_estimators=600, min_child_weight=4, max_depth=9,
                             learning_rate=0.2, gamma=0.7, n_jobs=-1)
best_xgb = best_xgb.fit(X_train, y_train)

print('train:', best_xgb.score(X_train, y_train), 'test:', best_xgb.score(X_test, y_test))
end = timer()
print(f'{round(end-start)} seconds elapsed.')

[11:29:44] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
train: 0.9172378306061542 test: 0.9151642380834774
651 seconds elapsed.


In [27]:
y_pred = best_xgb.predict(X_test)
print(classification_report(y_test, y_pred, labels=best_xgb.classes_))

              precision    recall  f1-score   support

         <30       0.45      0.00      0.01      1373
       Other       0.92      1.00      0.96     14823

    accuracy                           0.92     16196
   macro avg       0.69      0.50      0.48     16196
weighted avg       0.88      0.92      0.88     16196



In [28]:
auc_score = roc_auc_score(y_test, best_xgb.predict_proba(X_test)[:,1])
print('AUC:', auc_score)

AUC: 0.6433788085178351
